In [100]:
from mlchartist.array_builder import full_dataset_randomised_arrays_, #full_dataset_randomised_arrays
from mlchartist.preprocessing import train_test_split, thresholds_encoding

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import joblib
import os

from tensorflow.keras import models

SyntaxError: trailing comma not allowed without surrounding parentheses (<ipython-input-100-fe5df286b191>, line 1)

# Preprocessing

## Importing NASDAQ_100 Files

In [2]:
# Reads Nasdaq 100 components

nasdaq100 = pd.read_csv('../../raw_data/metadata/nasdaq100.csv', header=None)

In [3]:
nasdaq100_list = list(nasdaq100.values.flatten())

In [13]:
# Samples n random companies from Nasdaq 100 list

rand_tickers = random.sample(nasdaq100_list, 5)

In [14]:
joined_df = pd.DataFrame()

# Takes nrows from history from each company from the sample
nrows = 3000

for ticker in rand_tickers:
    ticker_path = '../../raw_data/processed/' + ticker.strip().lower() + '.csv'
    ticker_df = pd.read_csv(ticker_path, nrows=nrows)
    joined_df = joined_df.append(ticker_df)
    

## Stocks in DataFrame

In [15]:
# Prints sample tickers list

for tick in rand_tickers:
    print(tick)

MXIM
UAL
ISRG
ULTA
MDLZ


## Threshold Encoding

In [16]:
joined_df['date'] = pd.to_datetime(joined_df['date'])

#FIVE_TR = 0.0006
#TEN_TR = 0.0012
#TWENTY_TR = 0.0024

# Drops 'ADI' and 'OBV' from input columns
# INPUT_COLS = ['RSI', 'Stochastic', 'Stochastic_signal', 'ADI','OBV', 'ATR', 'ADX', 'ADX_pos', 'ADX_neg', 'MACD', 'MACD_diff','MACD_signal', '1D_past_return', '5D_past_return', '10D_past_return']
INPUT_COLS = ['RSI', 'Stochastic', 'Stochastic_signal', 'ATR', 'ADX', 'ADX_pos', 'ADX_neg', 'MACD', 'MACD_diff','MACD_signal', '1D_past_return', '5D_past_return', '10D_past_return']

joined_df = thresholds_encoding(joined_df)

#joined_df['5D_return_bin'] = (joined_df['5TD_return'] >= FIVE_TR)
#joined_df['10D_return_bin'] = (joined_df['10TD_return'] >= TEN_TR)
#joined_df['20D_return_bin'] = (joined_df['20TD_return'] >= TWENTY_TR)

In [17]:
joined_df.head()

,ticker,date,RSI,Stochastic,Stochastic_signal,ADI,OBV,ATR,ADX,ADX_pos,...,MACD_signal,5TD_return,10TD_return,20TD_return,1D_past_return,5D_past_return,10D_past_return,5D_return_bin,10D_return_bin,20D_return_bin
0,MXIM,1990-05-11,55.827296,74.065342,74.447064,8.616392e+06,18118712,0.011748,24.707014,20.046340,...,0.001768,0.058847,0.161880,0.088371,-0.015229,0.015004,0.046206,1,1,1
1,MXIM,1990-05-14,59.059040,100.000000,91.355114,8.874201e+06,18376521,0.011459,26.813936,19.084843,...,0.002409,0.086432,0.100672,0.100672,0.015465,0.045881,0.045881,1,1,1
2,MXIM,1990-05-15,59.059040,100.000000,91.355114,8.874201e+06,18710802,0.010640,28.770363,19.084843,...,0.003048,0.158623,0.100672,0.086432,0.000000,0.000000,0.062386,1,1,1
3,MXIM,1990-05-16,59.059040,80.921232,93.640411,7.539277e+06,20045726,0.010380,31.141968,22.979727,...,0.003646,0.158623,0.086432,0.086432,0.000000,0.030701,0.062386,1,1,1
4,MXIM,1990-05-17,65.034232,100.000000,93.640411,9.042434e+06,21548883,0.010667,33.783844,25.717304,...,0.004408,0.126538,0.070192,0.056346,0.028481,0.028481,0.075669,1,1,1


## Data Visualisation

/home/bob/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pandas/plotting/_tools.py:307: MatplotlibDeprecationWarning: 
The rowNum attribute was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use ax.get_subplotspec().rowspan.start instead.
  layout[ax.rowNum, ax.colNum] = ax.get_visible()
/home/bob/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pandas/plotting/_tools.py:307: MatplotlibDeprecationWarning: 
The colNum attribute was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use ax.get_subplotspec().colspan.start instead.
  layout[ax.rowNum, ax.colNum] = ax.get_visible()
/home/bob/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/pandas/plotting/_tools.py:313: MatplotlibDeprecationWarning: 
The rowNum attribute was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use ax.get_subplotspec().rowspan.start instead.
  if not layout[ax.rowNum + 1, ax.colNum]:
/home/bo

In [18]:
# Plots distribution of the input df without removing outliers

joined_df[INPUT_COLS].hist(figsize=(20, 15), bins=50)
plt.show()

## `train_x`, `train_y`, `test_x`, `test_y` Arrays Building

In [19]:

TARGET_COLS=['10D_return_bin']
outlier_validation={'1D_past_return': [-0.3, 0.3]}

stride = 1


train_x, train_y, test_x, test_y, scaler = full_dataset_randomised_arrays_(joined_df, 
                                                                                stride=stride, 
                                                                                input_cols=INPUT_COLS, 
                                                                                outlier_threshold=1, 
                                                                                outlier_validation=outlier_validation, 
                                                                                check_train_outliers=True,
                                                                                check_test_outliers=False, 
                                                                                target_col=TARGET_COLS, 
                                                                                time_window=30,
                                                                                test_set_size='2Y')

print('')
print('')
print('### Stats ###')
print('train_x', train_x.shape)
print('train_y', train_y.shape)
print('test_x', test_x.shape)
print('test_y', test_y.shape)
print('scaler', scaler)

print('')
print('')
print('### Validation ###')


5 Companies in Dataset
Starting MXIM: Company 1 of 5
Starting UAL: Company 2 of 5
Starting ISRG: Company 3 of 5
Starting ULTA: Company 4 of 5
Starting MDLZ: Company 5 of 5
All Companies Completed

Processing Stats: {'MXIM': {'train_possible_windows': 2687.0, 'train_outliers': 29, 'train_windows': 2658, 'test_possible_windows': 313.0, 'test_outliers': 29, 'test_windows': 284}, 'UAL': {'train_possible_windows': 2721.0, 'train_outliers': 123, 'train_windows': 2598, 'test_possible_windows': 279.0, 'test_outliers': 29, 'test_windows': 250}, 'ISRG': {'train_possible_windows': 2698.0, 'train_outliers': 59, 'train_windows': 2639, 'test_possible_windows': 302.0, 'test_outliers': 29, 'test_windows': 273}, 'ULTA': {'train_possible_windows': 2531.0, 'train_outliers': 29, 'train_windows': 2502, 'test_possible_windows': 469.0, 'test_outliers': 29, 'test_windows': 440}, 'MDLZ': {'train_possible_windows': 2548.0, 'train_outliers': 29, 'train_windows': 2519, 'test_possible_windows': 452.0, 'test_outlie

## `X_train` & `y_train` Arrays Random Sampling

In [20]:
# Samples n rows of train_x

indx = list(range(len(train_x)))
sample_indx = random.sample(indx, 10000)
X_train_sample =  train_x[[sample_indx], :][0]
y_train_sample = train_y[[sample_indx]]

/home/bob/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  


In [21]:
y_train_sample.shape

(10000, 1)

In [22]:
test_y.shape

(1670, 1)

# Modeling

## Model (commented)

from tensorflow.keras.metrics import Precision
from tensorflow.keras import regularizers
from tensorflow.keras import Sequential
from tensorflow.keras import layers, models 
from tensorflow.keras.optimizers import RMSprop, Adam, Adamax

optim = RMSprop(learning_rate=0.0001)
precision = Precision()

def init_model():
    model = Sequential()
    reg_l1 = regularizers.l1(0.001)
    reg_l2 = regularizers.l2(0.001)
    reg_l1_l2 = regularizers.l1_l2(l1=0.001, l2=0.001)
    model.add(layers.LSTM(200, return_sequences=True, input_shape=(30,13), activation='tanh'))
    model.add(layers.LSTM(200, activation='tanh'))
    model.add(layers.Dropout(0.3))
    #model.add(layers.BatchNormalization())
    model.add(layers.Dense(200, activation='relu', kernel_regularizer=reg_l1))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(100, activation='relu', bias_regularizer=reg_l2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(50, activation='relu', activity_regularizer=reg_l1_l2))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=[precision, 'accuracy'])
    
    return model

model = init_model()

from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(train_x, train_y, 
          epochs=500, 
          batch_size=16,
          validation_split=0.2,
          callbacks=[es])

## Balance of `train_y` and `test_y`

In [23]:
train_y.sum()/len(train_y)

0.5587643233199133

In [24]:
test_y.sum()/len(test_y)

0.5383233532934132

## Trained Model Loading

In [25]:
!ls ../../models/

Nasdaq_100_150k  Nasdaq_random_50  scaler


In [29]:
model = models.load_model('../../models/Nasdaq_100_150k/')

In [30]:
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_46 (LSTM)               (None, 30, 200)           171200    
_________________________________________________________________
lstm_47 (LSTM)               (None, 200)               320800    
_________________________________________________________________
dropout_69 (Dropout)         (None, 200)               0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 200)               800       
_________________________________________________________________
dense_92 (Dense)             (None, 200)               40200     
_________________________________________________________________
dropout_70 (Dropout)         (None, 200)               0         
_________________________________________________________________
dense_93 (Dense)             (None, 100)             

## Model evaluation

In [34]:
results = model.evaluate(test_x, test_y)
results

53/53 [==============================] - 2s 34ms/step - loss: 0.7501 - precision_24: 0.7096 - accuracy: 0.6587


[0.7501091957092285, 0.709554135799408, 0.658682644367218]

In [36]:
results[0]

0.7501091957092285

# Results Saving

## Current results in `df`

In [91]:
df = pd.DataFrame(columns=['n_companies', 'companies_list', 'benchmark', 'loss', 'precision', 'accuracy'])

In [92]:
#len(rand_tickers)

In [93]:
df['n_companies'] = int(len(rand_tickers))
df['companies_list'] = [', '.join(rand_tickers)]
df['benchmark'] = test_y.sum()/len(test_y)
df['loss'] = results[0]
df['precision'] = results[1]
df['accuracy'] = results[2]

In [94]:
df

,n_companies,companies_list,benchmark,loss,precision,accuracy
0,NaN,"MXIM, UAL, ISRG, ULTA, MDLZ",0.538323,0.750109,0.709554,0.658683


## Loading Previous Results

if 'charles_NASDAQ100_model_results' in os.listdir('stock_model_results/'):
    previous_results_df = joblib.load('stock_model_results/charles_NASDAQ100_model_results')
else:
    previous_results_df = pd.DataFrame(columns=['n_companies', 'companies_list', 'benchmark', 'loss', 'precision', 'accuracy'])

## Saving Current Results in New File on the Disk

saved_results_df = previous_results_df.append(df)

joblib.dump(saved_results_df,'stock_model_results/charles_NASDAQ100_model_results')

saving_process_test_df = joblib.load('stock_model_results/charles_NASDAQ100_model_results')
saving_process_test_df

# Testing Function 

In [117]:
from mlchartist.array_builder import full_dataset_randomised_arrays_#, full_dataset_randomised_arrays
from mlchartist.preprocessing import train_test_split, thresholds_encoding

import pandas as pd
import numpy as np
import random
import joblib
import os

from tensorflow.keras import models
from termcolor import colored

def model_testing_function(n_companies=5, n_test=25, trained_model_path='', rand=True, stocks_to_train=[],
                           nrows = 3000, verbose=0, new_save=False,
                           load_save_path='stock_model_results/charles_NASDAQ100_model_results',
                           save_path='stock_model_results/charles_NASDAQ100_model_results',
                           r5d=0.0006, same_thresholds=True, r10d=0.0012, r20d=0.0024, INPUT_COLS=[],
                           TARGET_COLS=['10D_return_bin'], outlier_validation={'1D_past_return': [-0.3, 0.3]},
                           stride = 1):
    """
    Test `n_test` times the trained model saved for `n_companies`
    
    If  `rand` set to False, test on the `stocks_to_train` list
    
    You have access to some of the function parameters, see doc of the following functions:
        -`thresholds_encoding()`
        -`full_dataset_randomised_arrays_()`
        
        (see below to see other default values of `full_dataset_randomised_arrays_()`)
    
    Other parameters ar
    
    If you leave INPUT_COLS empty, its default value will be:
        ['RSI', 'Stochastic', 'Stochastic_signal', 'ATR', 'ADX',
        'ADX_pos', 'ADX_neg', 'MACD', 'MACD_diff','MACD_signal',
        '1D_past_return', '5D_past_return', '10D_past_return']
    
    Other parameters of `full_dataset_randomised_arrays_()`:
        -outlier_threshold=1
        -check_train_outliers=True
        -check_test_outliers=False
        -time_window=30
        -test_set_size='2Y'
    
    """
    
    # Importing ticker list
    nasdaq100 = pd.read_csv('../../raw_data/metadata/nasdaq100.csv', header=None)
    nasdaq100_list = list(nasdaq100.values.flatten())
        
    
    joined_df = pd.DataFrame()

    for i in range(n_test):
        
        # Randomly select the stocks
        rand_tickers = random.sample(nasdaq100_list, n_companies)
        
        
        # Takes nrows from history from each company from the sample
        joined_df = pd.DataFrame()
        for ticker in rand_tickers:
            ticker_path = '../../raw_data/processed/' + ticker.strip().lower() + '.csv'
            ticker_df = pd.read_csv(ticker_path, nrows=nrows)
            joined_df = joined_df.append(ticker_df)

        if verbose ==2:
            for tick in rand_tickers:
                print(tick)

        joined_df['date'] = pd.to_datetime(joined_df['date'])

        # Drops 'ADI' and 'OBV' from input columns
        # INPUT_COLS = ['RSI', 'Stochastic', 'Stochastic_signal', 'ADI','OBV', 'ATR', 'ADX', 'ADX_pos', 'ADX_neg', 'MACD', 'MACD_diff','MACD_signal', '1D_past_return', '5D_past_return', '10D_past_return']
        if len(INPUT_COLS) == 0:
            INPUT_COLS = ['RSI', 'Stochastic', 'Stochastic_signal', 'ATR', 'ADX', 'ADX_pos', 'ADX_neg', 'MACD', 'MACD_diff','MACD_signal', '1D_past_return', '5D_past_return', '10D_past_return']

        joined_df = thresholds_encoding(joined_df, r5d=r5d, same_thresholds=same_thresholds, r10d=r10d, r20d=r20d)
        
        print('full_dataset_randomised_arrays_ verbose():')
        train_x, train_y, test_x, test_y, scaler = full_dataset_randomised_arrays_(joined_df, 
                                                                                        stride=stride, 
                                                                                        input_cols=INPUT_COLS, 
                                                                                        outlier_threshold=1, 
                                                                                        outlier_validation=outlier_validation, 
                                                                                        check_train_outliers=True,
                                                                                        check_test_outliers=False, 
                                                                                        target_col=TARGET_COLS, 
                                                                                        time_window=30,
                                                                                        test_set_size='2Y')

        if verbose == 2: 
            print('')
            print('')
            print('### Stats ###')
            print('train_x', train_x.shape)
            print('train_y', train_y.shape)
            print('test_x', test_x.shape)
            print('test_y', test_y.shape)
            print('scaler', scaler)

            print('')
            print('')
            print('### Validation ###')
            
        # Samples n rows of train_x
        indx = list(range(len(train_x)))
        sample_indx = random.sample(indx, 10000)
        X_train_sample =  train_x[[sample_indx], :][0]
        y_train_sample = train_y[[sample_indx]]

        if verbose >=1:
            print(f"train_y.sum()/len(train_y): {train_y.sum()/len(train_y)}")

        # Model loading
        model = models.load_model('../../models/Nasdaq_100_150k/')
        results = model.evaluate(test_x, test_y, verbose=0)

        if verbose >= 1:
            print(results)
        if verbose == 2:
            print(model.summary())

        # Current Results in `df`
        df = pd.DataFrame(columns=['n_companies', 'companies_list', 'benchmark', 'loss', 'precision', 'accuracy'])
        df['n_companies'] = int(len(rand_tickers))
        df['companies_list'] = [', '.join(rand_tickers)]
        df['benchmark'] = test_y.sum()/len(test_y)
        df['loss'] = results[0]
        df['precision'] = results[1]
        df['accuracy'] = results[2]

        if verbose >=1:
            print(df)
            print()

        # Loading previous results
        if 'charles_NASDAQ100_model_results' in os.listdir('stock_model_results/'):
            previous_results_df = joblib.load('stock_model_results/charles_NASDAQ100_model_results')
        else:
            previous_results_df = pd.DataFrame(columns=['n_companies', 'companies_list', 'benchmark', 'loss', 'precision', 'accuracy'])

        # Saving Current Results
        saved_results_df = previous_results_df.append(df)
        saved_results_df.reset_index(inplace=True)
        joblib.dump(saved_results_df,'stock_model_results/charles_NASDAQ100_model_results')
        
        print(colored(f"{i} iterations done on {len(n_test)}", 'green'))
        print()
        print()
    
    return saved_results_df

In [112]:
comprehensive_model_result = model_testing_function()

full_dataset_randomised_arrays_ verbose():
5 Companies in Dataset
Starting AVGO: Company 1 of 5
Starting ADBE: Company 2 of 5
Starting ASML: Company 3 of 5
Starting QCOM: Company 4 of 5
Starting GOOG: Company 5 of 5
All Companies Completed

Processing Stats: {'AVGO': {'train_possible_windows': 2334.0, 'train_outliers': 29, 'train_windows': 2305, 'test_possible_windows': 500.0, 'test_outliers': 29, 'test_windows': 471}, 'ADBE': {'train_possible_windows': 2592.0, 'train_outliers': 89, 'train_windows': 2503, 'test_possible_windows': 408.0, 'test_outliers': 29, 'test_windows': 379}, 'ASML': {'train_possible_windows': 2698.0, 'train_outliers': 59, 'train_windows': 2639, 'test_possible_windows': 302.0, 'test_outliers': 29, 'test_windows': 273}, 'QCOM': {'train_possible_windows': 2499.0, 'train_outliers': 119, 'train_windows': 2380, 'test_possible_windows': 501.0, 'test_outliers': 29, 'test_windows': 472}, 'GOOG': {'train_possible_windows': 1167.0, 'train_outliers': 29, 'train_windows': 1138,

/home/bob/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:109: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


65/65 [==============================] - 2s 33ms/step - loss: 1.0171 - precision_24: 0.6184 - accuracy: 0.5479
full_dataset_randomised_arrays_ verbose():
5 Companies in Dataset
Starting WDC: Company 1 of 5
Starting REGN: Company 2 of 5
Starting LRCX: Company 3 of 5
Starting EA: Company 4 of 5
Starting MELI: Company 5 of 5
All Companies Completed

Processing Stats: {'WDC': {'train_possible_windows': 2698.0, 'train_outliers': 29, 'train_windows': 2669, 'test_possible_windows': 302.0, 'test_outliers': 29, 'test_windows': 273}, 'REGN': {'train_possible_windows': 2697.0, 'train_outliers': 29, 'train_windows': 2668, 'test_possible_windows': 303.0, 'test_outliers': 29, 'test_windows': 274}, 'LRCX': {'train_possible_windows': 2682.0, 'train_outliers': 59, 'train_windows': 2623, 'test_possible_windows': 318.0, 'test_outliers': 29, 'test_windows': 289}, 'EA': {'train_possible_windows': 1734.0, 'train_outliers': 29, 'train_windows': 1705, 'test_possible_windows': 500.0, 'test_outliers': 29, 'test

/home/bob/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:109: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


53/53 [==============================] - 2s 38ms/step - loss: 0.7699 - precision_24: 0.7382 - accuracy: 0.6724
full_dataset_randomised_arrays_ verbose():
5 Companies in Dataset
Starting CSCO: Company 1 of 5
Starting LRCX: Company 2 of 5
Starting FISV: Company 3 of 5
Starting AVGO: Company 4 of 5
Starting CTSH: Company 5 of 5
All Companies Completed

Processing Stats: {'CSCO': {'train_possible_windows': 2684.0, 'train_outliers': 29, 'train_windows': 2655, 'test_possible_windows': 316.0, 'test_outliers': 29, 'test_windows': 287}, 'LRCX': {'train_possible_windows': 2682.0, 'train_outliers': 59, 'train_windows': 2623, 'test_possible_windows': 318.0, 'test_outliers': 29, 'test_windows': 289}, 'FISV': {'train_possible_windows': 2688.0, 'train_outliers': 29, 'train_windows': 2659, 'test_possible_windows': 312.0, 'test_outliers': 29, 'test_windows': 283}, 'AVGO': {'train_possible_windows': 2334.0, 'train_outliers': 29, 'train_windows': 2305, 'test_possible_windows': 500.0, 'test_outliers': 29,

/home/bob/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:109: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


51/51 [==============================] - 2s 33ms/step - loss: 1.0502 - precision_24: 0.5539 - accuracy: 0.5047
full_dataset_randomised_arrays_ verbose():
5 Companies in Dataset
Starting NVDA: Company 1 of 5
Starting CTX: Company 2 of 5
Starting INTC: Company 3 of 5
Starting BID: Company 4 of 5
Starting BIIB: Company 5 of 5
All Companies Completed

Processing Stats: {'NVDA': {'train_possible_windows': 2720.0, 'train_outliers': 209, 'train_windows': 2511, 'test_possible_windows': 280.0, 'test_outliers': 29, 'test_windows': 251}, 'CTX': {'train_possible_windows': 2747.0, 'train_outliers': 119, 'train_windows': 2628, 'test_possible_windows': 253.0, 'test_outliers': 29, 'test_windows': 224}, 'INTC': {'train_possible_windows': 2569.0, 'train_outliers': 249, 'train_windows': 2320, 'test_possible_windows': 431.0, 'test_outliers': 29, 'test_windows': 402}, 'BID': {'train_possible_windows': 2587.0, 'train_outliers': 59, 'train_windows': 2528, 'test_possible_windows': 413.0, 'test_outliers': 29, 

/home/bob/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:109: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


53/53 [==============================] - 2s 32ms/step - loss: 1.0642 - precision_24: 0.5428 - accuracy: 0.5245
full_dataset_randomised_arrays_ verbose():
5 Companies in Dataset
Starting KLAC: Company 1 of 5
Starting SWK: Company 2 of 5
Starting HA: Company 3 of 5
Starting EXPE: Company 4 of 5
Starting M: Company 5 of 5
All Companies Completed

Processing Stats: {'KLAC': {'train_possible_windows': 2688.0, 'train_outliers': 29, 'train_windows': 2659, 'test_possible_windows': 312.0, 'test_outliers': 29, 'test_windows': 283}, 'SWK': {'train_possible_windows': 2698.0, 'train_outliers': 89, 'train_windows': 2609, 'test_possible_windows': 302.0, 'test_outliers': 29, 'test_windows': 273}, 'HA': {'train_possible_windows': 2504.0, 'train_outliers': 29, 'train_windows': 2475, 'test_possible_windows': 496.0, 'test_outliers': 29, 'test_windows': 467}, 'EXPE': {'train_possible_windows': 2598.0, 'train_outliers': 29, 'train_windows': 2569, 'test_possible_windows': 402.0, 'test_outliers': 29, 'test_wi

/home/bob/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:109: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


54/54 [==============================] - 2s 37ms/step - loss: 0.9786 - precision_24: 0.5915 - accuracy: 0.5478
full_dataset_randomised_arrays_ verbose():
5 Companies in Dataset
Starting EXC: Company 1 of 5
Starting VRSN: Company 2 of 5
Starting AMD: Company 3 of 5
Starting FISV: Company 4 of 5
Starting AAL: Company 5 of 5
All Companies Completed

Processing Stats: {'EXC': {'train_possible_windows': 2495.0, 'train_outliers': 29, 'train_windows': 2466, 'test_possible_windows': 505.0, 'test_outliers': 29, 'test_windows': 476}, 'VRSN': {'train_possible_windows': 2712.0, 'train_outliers': 89, 'train_windows': 2623, 'test_possible_windows': 288.0, 'test_outliers': 29, 'test_windows': 259}, 'AMD': {'train_possible_windows': 2695.0, 'train_outliers': 89, 'train_windows': 2606, 'test_possible_windows': 305.0, 'test_outliers': 29, 'test_windows': 276}, 'FISV': {'train_possible_windows': 2688.0, 'train_outliers': 29, 'train_windows': 2659, 'test_possible_windows': 312.0, 'test_outliers': 29, 'tes

/home/bob/.pyenv/versions/3.7.7/envs/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:109: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


56/56 [==============================] - 2s 33ms/step - loss: 1.1389 - precision_24: 0.5466 - accuracy: 0.5184
full_dataset_randomised_arrays_ verbose():
5 Companies in Dataset
Starting KLAC: Company 1 of 5
Starting INT: Company 2 of 5
Starting AMD: Company 3 of 5
Starting KHC: Company 4 of 5
Starting PYPL: Company 5 of 5
All Companies Completed

Processing Stats: {'KLAC': {'train_possible_windows': 2688.0, 'train_outliers': 29, 'train_windows': 2659, 'test_possible_windows': 312.0, 'test_outliers': 29, 'test_windows': 283}, 'INT': {'train_possible_windows': 2684.0, 'train_outliers': 59, 'train_windows': 2625, 'test_possible_windows': 316.0, 'test_outliers': 29, 'test_windows': 287}, 'AMD': {'train_possible_windows': 2695.0, 'train_outliers': 89, 'train_windows': 2606, 'test_possible_windows': 305.0, 'test_outliers': 29, 'test_windows': 276}, 'KHC': {'train_possible_windows': 847.0, 'train_outliers': 29, 'train_windows': 818, 'test_possible_windows': 500.0, 'test_outliers': 29, 'test_w

ValueError: Sample larger than population or is negative

In [115]:
comprehensive_model_result_broken = joblib.load('stock_model_results/charles_NASDAQ100_model_results')

In [116]:
comprehensive_model_result_broken

,n_companies,companies_list,benchmark,loss,precision,accuracy
0,NaN,"ALGN, INCY, SNPS, WDAY, XLNX",0.571779,0.949156,0.637421,0.587730
0,NaN,"NXPI, WDAY, AMZN, ROST, ALGN",0.591757,0.990161,0.618377,0.525202
0,NaN,"AVGO, ADBE, ASML, QCOM, GOOG",0.579380,1.017072,0.618362,0.547919
0,NaN,"WDC, REGN, LRCX, EA, MELI",0.560803,0.769861,0.738239,0.672373
0,NaN,"CSCO, LRCX, FISV, AVGO, CTSH",0.541485,1.050175,0.553936,0.504679
0,NaN,"NVDA, CTXS, INTC, BIDU, BIIB",0.514354,1.064222,0.542819,0.524522
0,NaN,"KLAC, SWKS, HAS, EXPE, MU",0.537923,0.978591,0.591532,0.547841
0,NaN,"EXC, VRSN, AMD, FISV, AAL",0.530878,1.138875,0.546624,0.518414
